In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import os

# URLs for Andhra Pradesh and Odisha State Assembly Elections
urls = {
    "Andhra Pradesh": "https://results.eci.gov.in/AcResultGenJune2024/partywiseresult-S01.htm",
    "Odisha": "https://results.eci.gov.in/AcResultGenJune2024/partywiseresult-S18.htm"
}

parent_directory = "state-assembly-elections"
if not os.path.exists(parent_directory):
    os.makedirs(parent_directory)

def scrape_state_assembly_data(url, state_name):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    data = []
    table = soup.find("table", {"class": "table-party"})
    
    if table:
        headers = [header.text.strip() for header in table.find_all("th")]
        rows = table.find_all("tr")
        for row in rows:
            cols = row.find_all("td")
            if cols:
                data.append([col.text.strip() for col in cols])
        return pd.DataFrame(data, columns=headers)
    else:
        print(f"No table found for {state_name} on {url}")
        return pd.DataFrame()  # Return an empty DataFrame if no table is found

def save_charts(df, state_name):
    directory = os.path.join(parent_directory, state_name)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    if not df.empty:
        # Save to Excel
        excel_path = os.path.join(directory, f"{state_name}_assembly_results.xlsx")
        df.to_excel(excel_path, index=False)
        
        # Bar chart
        bar_chart_path = os.path.join(directory, f"{state_name}_party_distribution_bar.png")
        df.plot(kind="bar", x="Party", y="Seats", title=f"Party-wise Seat Distribution ({state_name})")
        plt.ylabel("Seats Won")
        plt.savefig(bar_chart_path)
        plt.close()
        
        # Pie chart
        pie_chart_path = os.path.join(directory, f"{state_name}_party_distribution_pie.png")
        df.plot(kind="pie", y="Seats", labels=df["Party"], autopct="%1.1f%%", title=f"Party-wise Seat Distribution ({state_name})")
        plt.ylabel("")
        plt.savefig(pie_chart_path)
        plt.close()
    else:
        print(f"No data found for {state_name}")

for state_name, url in urls.items():
    df_state_assembly = scrape_state_assembly_data(url, state_name)
    save_charts(df_state_assembly, state_name)
    print(f"Data and charts saved for {state_name}")

print("Scraping and chart generation completed.")


No table found for Andhra Pradesh on https://results.eci.gov.in/AcResultGenJune2024/partywiseresult-S01.htm
No data found for Andhra Pradesh
Data and charts saved for Andhra Pradesh
No table found for Odisha on https://results.eci.gov.in/AcResultGenJune2024/partywiseresult-S18.htm
No data found for Odisha
Data and charts saved for Odisha
Scraping and chart generation completed.
